In [4]:
import pandas as pd

In [5]:
# Read in data
pluspath = '../output/song-data-plus.csv'

data = pd.read_csv(pluspath)
data.head()

,year,rank,song,artist,gender,lyrics,femflag,mascflag,loveflag,genderref,femphases,mascphases,lovephases
0,2006,100,For You I Will (Confidence),Teddy Geiger,nb,im wandering the streets in a world underneat...,0,0,0,No reference,NaN,NaN,NaN
1,2006,66,Smack That,Akon Featuring Eminem,man,shady konvict upfront akon slim shady i see t...,1,1,1,Masc & fem reference,three block wheel style like whee girl i can t...,your gal is game in fact he is the one singing...,them rollin women just hoein big booty rollin ...
2,2006,82,Hate Me,Blue October,man,i have to block out thoughts of you so i dont...,1,1,1,Masc & fem reference,it used to be and then she whispered how can y...,had made and like a baby boy i never was a man...,is the one thing i wont touch again in a sick ...
3,2006,71,Who Says You Can't Go Home,Bon Jovi,man,i spent twenty years trying to get out of thi...,0,1,0,Same-gender,NaN,you take the home from the boy but not the boy...,NaN
4,2006,36,Ms. New Booty,Bubba Sparxxx Featuring Ying Yang Twins & Mr. ...,man,colipark bubba sparxxx booty booty booty boot...,1,1,1,Masc & fem reference,and you aint gotta sell sex girl it sells itse...,nothing else yeah im a country boy but that bi...,and da brothers gonna sho some love do that mo...


In [6]:
# Overall category counts
pd.crosstab(data.genderref.fillna('missing'), data.loveflag, margins=True)

loveflag,0,1,All
genderref,,,
Masc & fem reference,61,460,521
Mixed/nb - fem reference only,3,38,41
Mixed/nb - masc reference only,4,20,24
No reference,87,265,352
Opposite-gender,24,196,220
Same-gender,24,75,99
All,203,1054,1257


In [7]:
# Category counts by gender
pd.crosstab(data.genderref.fillna('missing'), data.gender, margins=True)

gender,man,mixed,nb,woman,All
genderref,,,,,
Masc & fem reference,334,100,1,86,521
Mixed/nb - fem reference only,0,38,3,0,41
Mixed/nb - masc reference only,0,22,2,0,24
No reference,161,75,8,108,352
Opposite-gender,182,0,0,38,220
Same-gender,66,0,0,33,99
All,743,235,14,265,1257


In [8]:
# Category counts by gender - Love songs only
lovedata = data[data['loveflag'] == 1]
lovedata.shape

pd.crosstab(lovedata.genderref.fillna('missing'), lovedata.gender, margins=True)

gender,man,mixed,nb,woman,All
genderref,,,,,
Masc & fem reference,293,93,1,73,460
Mixed/nb - fem reference only,0,35,3,0,38
Mixed/nb - masc reference only,0,18,2,0,20
No reference,111,63,7,84,265
Opposite-gender,160,0,0,36,196
Same-gender,49,0,0,26,75
All,613,209,13,219,1054


In [9]:
# Proportions
pd.crosstab(lovedata.genderref.fillna('missing'), lovedata.gender, margins=True, normalize='columns')

gender,man,mixed,nb,woman,All
genderref,,,,,
Masc & fem reference,0.477977,0.444976,0.076923,0.333333,0.436433
Mixed/nb - fem reference only,0.000000,0.167464,0.230769,0.000000,0.036053
Mixed/nb - masc reference only,0.000000,0.086124,0.153846,0.000000,0.018975
No reference,0.181077,0.301435,0.538462,0.383562,0.251423
Opposite-gender,0.261011,0.000000,0.000000,0.164384,0.185958
Same-gender,0.079935,0.000000,0.000000,0.118721,0.071157


Interesting? things (while knowing these are ROUGH flags):
* Women & NB more "No reference" songs
* Women high proportion "Same-gender" songs
* Almost half of the men artist songs have both "Masc & fem reference" compared to 1/3 of the women artist songs

In [25]:
# Additional features to whittle down multi-reference & same-gender songs
# Test if can weed out self-gender references
# lol of course phrases is misspelled in the orig data..

firstflags = ['i','im','id','ill','ive','me','my','mine','myself']

def selfreftest(row):
    '''
    '''
    if (row['genderref'] == "Masc & fem reference") or (row['genderref'] == "Same-gender"):
        if row['gender'] == 'man':
            phrases = row['mascphases'].split(', ')
            for p in phrases:
                tokens = p.split(" ")
                print(tokens)
                firstintersect = [w for w in tokens if w in firstflags]
                if len(firstintersect) > 0:
                    print("True self ref?")
            print()
#         if row['gender'] == 'woman':
#             phrases = row['femphases'].split(', ')
#             for p in phrases:
#                 tokens = p.split(" ")
#                 print(tokens)
#                 firstintersect = [w for w in tokens if w in firstflags]
#                 if len(firstintersect) > 0:
#                     print("True self ref?")
#             print()
    return ""

data['selfreftest'] = data.apply(selfreftest, axis=1)

# Not as clean as I was hoping... Prob should have replaced line breaks with .'s when processing so could have "sentences"
# and could try to POS tag? Although songs use more colorful language so idk how helpful that would have been either

# I think it might be breaking cause the phrase extraction isnt perfect - there are 13 (inc love phrases) that it
# didnt pull out b/c of numbers or other odd characters in the surrounding words. But gets through enough for testing

['your', 'gal', 'is', 'game', 'in', 'fact', 'he', 'is', 'the', 'one', 'singing', 'the', 'song']

['had', 'made', 'and', 'like', 'a', 'baby', 'boy', 'i', 'never', 'was', 'a', 'man', 'until']
True self ref?

['you', 'take', 'the', 'home', 'from', 'the', 'boy', 'but', 'not', 'the', 'boy', 'from', 'his']
['of', 'their', 'own', 'just', 'a', 'hometown', 'boy', 'born', 'a', 'rolling', 'stone', 'who', 'says']

['nothing', 'else', 'yeah', 'im', 'a', 'country', 'boy', 'but', 'that', 'big', 'city', 'bottom', 'fill']
True self ref?
['things', 'i', 'once', 'was', 'a', 'breast', 'man', 'now', 'it', 'seems', 'ever', 'since', 'i']
True self ref?
['put', 'his', 'hand', 'on', 'it', 'then', 'see', 'what', 'it', 'do']

['in', 'your', 'whip', 'nigga', 'always', 'do', 'his', 'thing', 'bitch', 'locking', 'the', 'strip', 'come']

['up', 'right', 'behind', 'and', 'its', 'in', 'his', 'throat', 'windows', 'down', 'gotta', 'stop', 'pollution']
['swerving', 'all', 'up', 'in', 'the', 'curve', 'man', 'nigga', 'been'

['same', 'not', 'sayin', 'im', 'the', 'richest', 'man', 'alive', 'but', 'im', 'in', 'the', 'game']
True self ref?
['us', 'that', 'white', 'sand', 'surrounding', 'us', 'he', 'be', 'handcuffing', 'he', 'should', 'work', 'for']

['you', 'then', 'cause', 'im', 'a', 'ramblin', 'man', 'i', 'aint', 'ever', 'gonna', 'change', 'i']
True self ref?
['night', 'is', 'black', 'as', 'the', 'coffee', 'he', 'was', 'drinkin', 'and', 'in', 'the', 'waitress']
['love', 'is', 'strong', 'enough', 'to', 'make', 'him', 'stay', 'shes', 'answered', 'by', 'the', 'tail']
['shed', 'trade', 'colorado', 'if', 'hed', 'take', 'her', 'with', 'him', 'closes']
['lights', 'shining', 'through', 'the', 'window', 'pane', 'he', 'said', 'i', 'wanna', 'see', 'you', 'again']
True self ref?
['colorado', 'if', 'hed', 'take', 'her', 'with', 'him', 'closes', 'the', 'door', 'before', 'the', 'winter']
['sees', 'the', 'same', 'ol', 'light', 'shinin', 'he', 'thinks', 'of', 'colorado', 'and', 'the', 'girl']
['you', 'then', 'she', 'said', 

AttributeError: 'float' object has no attribute 'split'